In [5]:
# This is an example of how we see 
# the package work. The functions listed here
# are probably the only ones that should be exposed, ie documented.
# others should br prepended with a double underscore
#  
# The cognet directory has the following "modules"
# which are seprate .py files containing clases and functions
# The modules are cognet.py, dataFormatter.py, model.py, util.py, viz.py
# we will write the viz.py later.
print("running")
import sys
#sys.path.append("../cognet")
#import os 
#print(os.getcwd())

from quasinet.qnet import qdistance
from cognet.cognet import cognet as cg
#import importlib
#import cognet as Cg
#importlib.reload(Cg)

from cognet.dataFormatter import dataFormatter
from cognet.model import model 
#import cognet.util
import pandas as pd
import numpy as np
#from cognet.viz import distance_contour 

yr = '2018'
POLEFILE='examples_data/polar_vectors.csv'
QPATH='examples_data/gss_'+yr+'.joblib'
IMMUTABLE_FILE='examples_data/immutable.csv'
GSSDATA = 'examples_data/gss_'+yr+'.csv'

running


In [6]:
# ---------------------------------
# testing dataFormatter
data = dataFormatter(samples=GSSDATA,
                test_size=0.5)
# load the sample data
# have option for test/train split
# make checks to ensure we will not bark at qnet construction 
# data.train() returns traininh data
# data.test() returns test data
print(data.samples[:2])
features,samples = data.train()
# we can set mutable and immutable vars from list or file
im_vars_df = pd.read_csv(IMMUTABLE_FILE, names=['vars'])
im_vars_list = im_vars_df.vars.to_list()
mutable_vars, immutable_vars = data.mutable_variables(immutable_list=im_vars_list)
mutable_vars, immutable_vars = data.mutable_variables(IMMUTABLE_FILE=IMMUTABLE_FILE)
# -------------------------------------

            wrkstat HRS1 HRS2 evwork        wrkslf  wrkgovt OCC10 PRESTG10  \
0  temp not working    e    c    NaN  someone else  private     b        c   
1  working fulltime    c    e    NaN  someone else  private     b        d   

  PRESTG105PLUS INDUS10  ...    neisafe rlooks rgroomed rweight rhlthend wtss  \
0             c       c  ...  very safe    NaN      NaN     NaN      NaN    e   
1             d       c  ...  very safe    NaN      NaN     NaN      NaN    c   

  wtssnr wtssall vstrat vpsu  
0      e       e   3301    1  
1      c       c   3301    1  

[2 rows x 1034 columns]


In [7]:
# -------------------------------------
# testing model functionality
# can either input features and samples directly, or infer from data obj
model_ = model()

# qnet construction parameters
test_model_buildqnet = False
# infer qnet
if test_model_buildqnet:
        model_.fit(data_obj=data)
        model_.export_dot("tmp_dot_modelclass.dot",
                        generate_trees=True)
        model_.save("tmp_nodelclass.joblib")
        #model_.load("tmp_nodelclass.joblib")
else:
    model_.load("examples_data/gss_2018.joblib")
# -------------------------------------

updating


In [8]:
# -------------------------------------
# testing cognet
# set some paramaters in instantiating cognet class 
# if loading from model obj, no need to load_data, otherwise, load_data

Cg = cg()
print(len(model_.features))
Cg.load_from_model(model_, samples_file='examples_data/gss_2018.csv')

# distance calculation for individual samples    
# we have a nsteps parameter (for sample 1 and sample2)
# which qsamples the sample1 and sample2 if set before
# computing distance. Note qsampling must only 
# change mutable varaibles, so need to compute base-freq
distance = Cg.distance(samples[1],samples[3],nsteps1=5, nsteps2=5)
print(distance)
qdistance_ = qdistance(samples[1],samples[3],Cg.qnet,Cg.qnet)
print("actual:{}".format(qdistance_))
#distance = Cg.distance(data.samples[0],data.samples[1])
#distance = Cg.distance(data.test[0],data.test[1])
# -------------------------------------

1034
0.10077681086456598
actual:0.09990742615579429


In [ ]:
#------------------
# produce stats on how many column names actually match
stats = Cg.set_poles(POLEFILE,"R","L",steps=120)

# compute polar distance matrix
dmatrix = Cg.polar_separation(nsteps=0)
#------------------

In [ ]:
#------------------
# the following are for single samples
dissonance_array = Cg.dissonance(1)
returndict = {}
rederr,r_prob,rand_err,s,qs,s_rand,mask_ = Cg.randomMaskReconstruction(1, returndict)# sample=np.array(samples[1]))
print(rederr,r_prob,rand_err,s,qs,s_rand,mask_)
#ideology_index = Cg.compute_DLI_sample(3)
Cg.num_qsamples = 5
ideology_index = Cg.ideology(3,pole_1="R",pole_2="L")

# get dispersion of an individual sample
Dispersion_ = Cg.dispersion(3)
print(Dispersion_)
# compute distance from each pole
array_distances = Cg.polarDistance(1, returndict)
#-------------------

In [ ]:
#-------------------
# the following are for arrays of samples
# multiprocessing suffices
count = 0
if count == 0:
    Cg.set_nsamples(10)
    count +=1
# computing polar_indices makes sure that dissonance matrix only takes in polar cols
Cg.compute_polar_indices()
dissonance_array = Cg.dissonance_matrix(output_file='examples_results/DISSONANCE_matrix.csv')
# multiprocessing suffices
dataframes,error_array = Cg.randomMaskReconstruction_multiple('examples_results/randomMaskRecon_test.csv')
# multiprocessing suffices
ideology_index = Cg.compute_DLI_samples('ideology','examples_results/ideology.csv')
# multiprocessing suffices
local_dispersion = Cg.compute_DLI_samples('dispersion', 'examples_results/dispersion_test.csv')
# compute distance from each pole
# multiprocessing suffices
array_distances = Cg.polarDistance_multiple('examples_results/polarDistance_multiple_test.csv')
#-------------------

In [ ]:
#the following must use parallelization
# next one must use mpi and hence will not run
# with mpi without maybe a seprate script.
# But look here: https://stackoverflow.com/questions/25772289/python-multiprocessing-within-mpi
distance_matrix=Cg.distfunc_multiples("examples_results/distfunc_multiples_testing.csv")


In [ ]:
## embedding
Cg.year = '2018'
Cg.embed('examples_data/DMAT_2018.csv', 'embed', 'examples_results/', pca_model=True)
#pd.read_csv('examples_results/embed_E_2018.csv')


In [ ]:
## generate pole separation across years
#creed2_/GSS/Qnets
import os
import matplotlib.pyplot as plt
import collections
import seaborn as sns 

# get years 
features_by_year = pd.read_csv('examples_data/features_by_year_GSS.csv',
                               keep_default_na=True, 
                               index_col=False)
qnets_by_year = {x[-11:-7]: x for x in os.listdir('../../creed2_/GSS/Qnets/')}
data_by_year = {x[4:8]: x for x in os.listdir('examples_data/processed_data')}
polarDist_by_year = {}
for year, qnet in qnets_by_year.items():
    qnet = '../../creed2_/GSS/Qnets/' + qnet
    model_ = model()
    model_.load(qnet)
    Cg = cognet()
    Cg.load_from_model(model_, samples_file='examples_data/processed_data/gss_2018.csv')
    # produce stats on how many column names actually match
    stats = Cg.set_poles(POLEFILE, steps=120)

    # compute polar distance matrix
    dmatrix = Cg.polar_separation(nsteps=0)
    polarDist_by_year[year]=dmatrix[0][1]
polarDist_by_year

In [ ]:
sorted_dist = collections.OrderedDict(sorted(polarDist_by_year.items()))
df = pd.DataFrame.from_dict(sorted_dist, orient='index')
df.reset_index(level=0, inplace=True)
df.columns = ['year', 'dist']

sns.set(style='darkgrid', rc={'figure.figsize':(20, 8)})
sns.lineplot(x='year', y='dist', data=df)
    
# year = list(sorted_dist.keys())
# dist = list(sorted_dist.values())     
# plt.plot(year, dist)
# plt.show